In [4]:
import os

for image_name in os.listdir('datasets/COCO/test2017'):
    if image_name.endswith('.jpg'):
        os.system(f'python tools/demo.py image -n yolox-s -c yolox_s.pth --path datasets/COCO/test2017/{image_name} --conf 0.25 --nms 0.45 --tsize 640 --save_result --device gpu')


KeyboardInterrupt: 

In [10]:
# load json file datasets/annotations/captions_val2017.json
# and save the image id and caption in a dataframe
import json
import pandas as pd
# JSON file
f = open ('datasets/annotations/instances_val2017.json', "r")

# Reading from file
data = json.loads(f.read())
data.keys()

In [7]:
data['annotations'][0]

In [8]:
data['images'][0]

{'license': 4,
 'file_name': '000000397133.jpg',
 'coco_url': 'http://images.cocodataset.org/val2017/000000397133.jpg',
 'height': 427,
 'width': 640,
 'date_captured': '2013-11-14 17:02:52',
 'flickr_url': 'http://farm7.staticflickr.com/6116/6255196340_da26cf2c9e_z.jpg',
 'id': 397133}

In [11]:
COCO_dataset_classes={0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microwave', 69: 'oven', 70: 'toaster', 71: 'sink', 72: 'refrigerator', 73: 'book', 74: 'clock', 75: 'vase', 76: 'scissors', 77: 'teddy bear', 78: 'hair drier', 79: 'toothbrush'}
from sahi.utils.coco import Coco, CocoCategory, CocoImage, CocoAnnotation
from sahi.utils.file import save_json
coco = Coco()
save_path = 'datasets/annotations/custom.json'
for i in range(80):
    coco.add_category(CocoCategory(id=i, name=COCO_dataset_classes[i]))
def add_image(img_info, output):
    coco_image = CocoImage(file_name=img_info['file_name'], height=img_info['height'], width=img_info['width'], id=img_info['id'])
    ratio = img_info["ratio"]
    img = img_info["raw_img"]
    if output is None:
        return img
    output = output.cpu()

    bboxes = output[:, 0:4]

    # preprocessing: resize
    bboxes /= ratio
    cls = output[:, 6]
    for i in range(len(bboxes)):
        coco_image.add_annotation(
            CocoAnnotation(
                bbox=bboxes[i],
                category_id=cls[i],
                category_name=coco.get_category_name(cls[i]),
            )
        )
    coco.add_image(coco_image)
    save_json(data=coco.json, save_path=save_path)

In [ ]:
def dataset2coco(df):

    annotion_id = 0

    annotations_json = {
        "info": [],
        "licenses": [],
        "categories": [],
        "images": [],
        "annotations": []
    }

    info = {
        "year": "2021",
        "version": "1",
        "description": "COTS dataset - COCO format",
        "contributor": "",
        "url": "https://kaggle.com",
        "date_created": "2022-01-29T15:01:26+00:00"
    }
    annotations_json["info"].append(info)

    lic = {
            "id": 1,
            "url": "",
            "name": "Unknown"
        }
    annotations_json["licenses"].append(lic)

    classes = {"id": 0, "name": "starfish", "supercategory": "none"}

    annotations_json["categories"].append(classes)


    for ann_row in df.itertuples():

        images = {
            "id": ann_row[0],
            "license": 1,
            "file_name": str(ann_row[5]) + '.jpg', # use the image id with video information
            "height": 720,
            "width": 1280,
            "date_captured": "2021-11-30T15:01:26+00:00"
        }

        annotations_json["images"].append(images)

        for bbox in ann_row.annotations:
            # some boxes in COTS are outside the image height and width
            if (bbox['x'] + bbox['width'] > 1280):
                b_width = bbox['x'] - 1280
            if (bbox['y'] + bbox['height'] > 720):
                b_height = bbox['y'] - 720

            image_annotations = {
                "id": annotion_id,
                "image_id": ann_row[0],
                "category_id": 0,
                "bbox": [bbox['x'], bbox['y'], bbox['width'], bbox['height']],
                "area": bbox['width'] * bbox['height'],
                "segmentation": [],
                "iscrowd": 0
            }

            annotion_id += 1
            annotations_json["annotations"].append(image_annotations)


    print(f"Dataset COTS annotation to COCO json format completed! Files: {len(df)}")
    return annotations_json

In [ ]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

# Path to your test2017.json file
annotation_file = 'path/to/test2017.json'

# Path to the ground truth annotations file for the COCO test dataset
gt_file = 'datasets/annotations/image_info_test2017.json'

# Create a COCO object for the ground truth annotations
coco_gt = COCO(gt_file)

# Load the predictions in COCO format
coco_dt = coco_gt.loadRes(annotation_file)

# Create a COCOeval object for evaluation
coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')

# Run the evaluation
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()